# Sub Question Query Engine

Often, we encounter scenarios where our queries span across multiple documents.

In this notebook, we delve into addressing complex queries that extend over various documents by breaking them down into simpler sub-queries and generate answers using the `SubQuestionQueryEngine`.

## Installation

In [ ]:
# !pip install llama-index -q
# !pip install llama-index-llms-anthropic -q
# !pip install llama-index-embeddings-huggingface -q

## Set Anthropic API Key

In [2]:
import os

os.environ["ANTHROPIC_API_KEY"] = "YOUR ANTHROPIC API KEY"

## Setup LLM and Embedding model

We will use anthropic latest released `Claude 3 Opus` models.

In [3]:
from llama_index.llms.anthropic import Anthropic
from llama_index.embeddings.huggingface import (
    HuggingFaceEmbedding,
)  # since Anthropic doesn't provide embedding models

In [ ]:
llm = Anthropic(temperature=0.0, model="claude-3-opus-20240229")
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

In [5]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

## Setup Logging

In [6]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.
import nest_asyncio

nest_asyncio.apply()

import logging
import sys

# Set up the root logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Set logger level to INFO

# Clear out any existing handlers
logger.handlers = []

# Set up the StreamHandler to output to sys.stdout (Colab's output)
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)  # Set handler level to INFO

# Add the handler to the logger
logger.addHandler(handler)

from IPython.display import display, HTML

## Download Data

We will use Uber and Lyft 2021 10K SEC Filings.

In [7]:
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/uber_2021.pdf' -O './uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/lyft_2021.pdf' -O './lyft_2021.pdf'

--2024-03-13 07:31:30--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘./uber_2021.pdf’

./uber_2021.pdf     100%[===================>]   1.79M  --.-KB/s    in 0.06s   

2024-03-13 07:31:30 (31.1 MB/s) - ‘./uber_2021.pdf’ saved [1880483/1880483]

--2024-03-13 07:31:30--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/lyft_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, 

## Load Data

In [8]:
from llama_index.core import SimpleDirectoryReader

lyft_docs = SimpleDirectoryReader(input_files=["lyft_2021.pdf"]).load_data()
uber_docs = SimpleDirectoryReader(input_files=["uber_2021.pdf"]).load_data()

In [9]:
print(f"Loaded lyft 10-K with {len(lyft_docs)} pages")
print(f"Loaded Uber 10-K with {len(uber_docs)} pages")

Loaded lyft 10-K with 238 pages
Loaded Uber 10-K with 307 pages


## Index Data

In [10]:
from llama_index.core import VectorStoreIndex

lyft_index = VectorStoreIndex.from_documents(lyft_docs[:100])
uber_index = VectorStoreIndex.from_documents(uber_docs[:100])

## Create Query Engines

In [11]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=5)

In [12]:
uber_engine = uber_index.as_query_engine(similarity_top_k=5)

## Querying

In [13]:
response = await lyft_engine.aquery(
    "What is the revenue of Lyft in 2021? Answer in millions with page reference"
)
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))

HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


In [14]:
response = await uber_engine.aquery(
    "What is the revenue of Uber in 2021? Answer in millions, with page reference"
)
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))

HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


## Create Tools

In [15]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine

query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description="Provides information about Lyft financials for year 2021",
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description="Provides information about Uber financials for year 2021",
        ),
    ),
]

## Create `SubQuestionQueryEngine`

In [16]:
sub_question_query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools
)

## Querying

In [17]:
response = await sub_question_query_engine.aquery(
    "Compare revenue growth of Uber and Lyft from 2020 to 2021"
)

HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
Generated 4 sub questions.
[uber_10k] Q: What was Uber's revenue in 2020?
[uber_10k] Q: What was Uber's revenue in 2021?
[lyft_10k] Q: What was Lyft's revenue in 2020?
[lyft_10k] Q: What was Lyft's revenue in 2021?
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 429 Too Many Requests"
Retrying request to /v1/messages in 0.783848 seconds
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 429 Too Many Requests"
Retrying request to /v1/messages in 0.887799 seconds
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 429 Too Many Requests"
Retrying request to /v1/messages in 1.882672 seconds
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 429 Too Many Requests"
Retrying request to /v1/messages in 1.956274 seconds
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 429 Too Many Requests"
Retrying request to /v1/messages in 3.167717 sec

In [18]:
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))

In [19]:
response = await sub_question_query_engine.aquery(
    "Compare the investments made by Uber and Lyft"
)

HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
Generated 2 sub questions.
[uber_10k] Q: What investments did Uber make in 2021
[lyft_10k] Q: What investments did Lyft make in 2021
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
[uber_10k] A: Based on the provided context, some key investments Uber made in 2021 include:

- Increasing incentives, discounts, and promotions to attract more Drivers, consumers, merchants, shippers, and carriers to their platform
- Expanding into existing or new markets
- Increasing research and development expenses  
- Expanding marketing channels and operations
- Hiring additional employees
- Adding new products and offerings to their platform

The context indicates Uber expected operating expenses to increase significantly as they continued to make these investments, even if it meant incurring losses in the near term. The goal was to increase usage of their platform and expand their business.
HTTP Requ

In [20]:
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))